<a href="https://colab.research.google.com/github/vinayastar01/deep-learning-2025/blob/main/day5_Transfer_learning_Scaling_Up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transfer Learning with TensorFlow Part 3: Scalling Up

In [2]:
import tensorflow as tf

In [4]:
!nvidia-smi

Sat Apr 12 06:54:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2025-04-12 08:53:24--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-04-12 08:53:25 (108 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

--2025-04-12 08:53:27--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.207, 172.253.118.207, 74.125.200.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  22.7MB/s    in 70s     

2025-04-12 08:54:37 (22.1 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [6]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir, compare_historys

In [7]:
unzip_data("101_food_classes_10_percent.zip")

In [8]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [9]:
walk_through_dir("101_food_classes_10_percent")

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent/train'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/panna_cotta'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/scallops'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/risotto'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/peking_duck'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/miso_soup'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/prime_rib'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/grilled_cheese_sandwich'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/hot_dog'.
There are 0 directories and 75 images in '101_foo

In [10]:
#Setup data inputs
IMG_SIZE=(224,224)
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                            label_mode = "categorical",
                                                                            image_size = IMG_SIZE,
)


Found 7575 files belonging to 101 classes.


In [11]:
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode ="categorical",
                                                                image_size = IMG_SIZE,
                                                                shuffle=False)

Found 25250 files belonging to 101 classes.


In [12]:
#Create checkpoint callback
checkpoint_path = "101_classes_10_percent_data_model_checkpoint.weights.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True)


In [13]:
 #Create data augmentation layer  to incorporate it right inot thr model

 from tensorflow.keras import layers

 from tensorflow.keras.models import Sequential
 from tensorflow.keras import layers


In [14]:
layers.Rescaling(1./255)
layers.RandomFlip("horizontal")
layers.RandomRotation(0.2)


<RandomRotation name=random_rotation, built=False>

In [26]:
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    "101_food_classes_10_percent",
    image_size=(224, 224)

)


Found 32825 files belonging to 2 classes.


In [28]:
#setup data augmentation

data_augmentation = Sequential([
    layers.Rescaling(1./255),
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.2),
], name = "data_augmentation")

In [29]:
#setup the base model and freeze its layers (this will extract features)

base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

#setup model architecture

inputs = layers.Input(shape=(224,224,3), name = "input-layer")
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name = "global_average_pooling_layer")(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names),activation="softmax", name ="output_layer")(x)
model = tf.keras.Model(inputs, outputs)


In [32]:
#Compile the model
model.compile( loss="sparse_categorical_crossentropy",
              optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])

In [35]:
model.fit(train_data_all_10_percent,
          epochs = 2,
          validation_data = test_data,
          validation_steps=int(0.15 * len(test_data)),
          callbacks = [checkpoint_callback])

Epoch 1/2
1026/1026 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7670 - loss: 0.5508

ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 101), output.shape=(None, 2)

In [34]:
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input-layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling_layer    │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 2)              │         2,562 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,057,259 (15.48 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

 Optimizer params: 5,126 (20.03 KB)